In [1]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt') # one time execution
import re
import glob
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import codecs

In [9]:
path = 'Disruptive Innovation Text\\1- Innovating in an Evolving World.txt' 
files = glob.glob(path)
# iterate over the list getting each file 
redundant_sentences = ['figure', 'Figure', 'Citigroup', 'Source', 'source']
articles_list = []
for fle in files:
    with codecs.open(fle, "r",encoding='utf-8', errors='ignore') as f:
    #with open(fle, 'rb') as f:
        article_content = ''
        for line in f:
            #line = line.replace('\r', '')
            #line = line.replace(' \n', '')
            for word in redundant_sentences:
                if word in line:
                    line = ''
            article_content += line
        articles_list.append(article_content)
articles_dict = {'article_text': articles_list}
df = pd.DataFrame.from_dict(articles_dict)

'\r\nInnovating in an Evolving \r\nWorld: \r\nSlow Change \r\nwith \r\nLong-Term Impact \r\n\r\nHumans are hard-wired to respond to instantaneous \r\nchange: Our fight or flight \r\nresponse evolved to make snap decisions based on \r\nimmediate danger. It is not in \r\nour nature to identify and react to challenges that arise slowly, even those with great \r\nlong-term \r\nimpact on \r\nour lives. \r\n\r\n\r\nBusinesses face a \r\nsimilar problem when dealing with \r\nslow change \r\nin \r\ntheir \r\nindustries. Large-scale societal, economic, and technological trends that emerge \r\ngradually and continuously over time can be \r\nall too easily \r\noverlooked. \r\nThat is \r\nbecause \r\nslow change \r\noften occurs outside \r\nof an organizations line-of-sight and \r\ncan \r\narise from the merger of many disparate \r\ndevelopments, each of which may be \r\nlost in the day-to-day \r\nnoise. \r\n\r\n\r\nEven the \r\nmost forward-thinking companies can \r\nmiss \r\nslow change \r\noccu

In [76]:
sentences = []
for s in df['article_text']:
  sentences.append(nltk.sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [68]:
sentences[:5]

['6. eSportsThe Rise of theProfessional Gamers\n\neSports (alsoknown as electronic sportsor professional video gaming) is aform ofcompetition using videogames.',
 'Most commonly, eSports take the formoforganized, multiplayer videogamecompetitionsbetween professional players.Similar to traditional sporting events liketheFIFAWorld Cup, sports fansgettogether to watch their favorite professional teams competewhile several millionsalso tune into watcha livestreamor highlightsof theevent.These video gametournamentscanbe playedacrossmany devices including PCs, consoles, andtablets, though PCs arepredominantly usedin tournamentsgiven their computingpower and speed.',
 'Many genres of gamesexist that can beplayed duringthesetournamentsincluding (1)First Person Shooter Games suchas Halo 2,(2)Multiplayer Online BattleArena (MOBA) gamessuch as Dota 2 and League ofLegends,(3)Strategy Gamessuch as Starcraft,(4)FightingGames,(5)SportsGames,and(5) Racing Games.',
 'The newest genreof eSports isSurviv

In [46]:
# Extract word vectors
word_embeddings = {}
f = open('./gloves.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [77]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [78]:
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [79]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [80]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [81]:
nx_graph = nx.from_numpy_matrix(sim_mat)
#scores = nx.pagerank(G=nx_graph, alpha=0.85, personalization=None, max_iter=200, tol=1e-08, nstart=None, weight='weight')
scores = nx.pagerank_numpy(nx_graph, alpha=0.85, personalization=None, weight='weight', dangling=None)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [82]:
# Extract top 10 sentences as the summary
for i in range(10):
    print(ranked_sentences[i][1])

Blue means that the language has been added in the past year.
Nature Medicine 2016).
21 Gartner Inc.
Who Gets Displaced?
Solid electrolytes were developed in the1970s, but insufficient ionic conductivity limited their application.
2018


August 2018 Citi GPS: Global Perspectives & Solutions Citi GPS: Global Perspectives & Solutions
43
3.
Right now there is a trade-off between(limiting output and volume degradation) controlling output degradation over time and improving energy density.Lowering costs Batteries currently account for about one-third of the total EV price.
Nature.
How Close to Reality?
We dont view it this way.
